# 3 - Coloring problem

#### Theorem (Planar graph 3-colorings, Penrose 1971)
The number K of proper 3- edge-colorings of a planar 3-regular graph is obtained by replacing each node with an order-3 **epsilon tensor**, replacing each edge with a wire, and then contracting the resulting tensor network.

> Roger Penrose, “Applications of negative dimensional tensors,” in Combinatorial Mathematics and its Applications, edited by D. Welsh (Academic Press, New York, 1971) pp. 221–244.

In [1]:
using OMEinsum, LinearAlgebra

In [31]:
# define the symmetrizer tensor
s_tensor(n::Int) = map(x->Int(length(unique(x.I)) == n), CartesianIndices(fill(n,n)|>Tuple))
I3 = Matrix(I, 3,3)
s = s_tensor(3)

3×3×3 Array{Int64,3}:
[:, :, 1] =
 0  0  0
 0  0  1
 0  1  0

[:, :, 2] =
 0  0  1
 0  0  0
 1  0  0

[:, :, 3] =
 0  1  0
 1  0  0
 0  0  0

## A two-node 3-regular graph


<img src="images/twonode.png" width="200px"/>

In [3]:
# einstein summation notation
ein"ijk,ijk->"(s, s)

0-dimensional Array{Int64,0}:
6

<img src="images/twonode_all.png" width="800px"/>

## Complete Graph $K_4$

In [4]:
ein"ijk,klm,njo,oli->"(s, s, s, s)

0-dimensional Array{Int64,0}:
6

## Peterson graph


<img src="images/_petersenijk.png" width="400px"/>

In [60]:
ein"afl,bhn,cjf,dlh,enj,ago,big,cki,dmk,eom->"(fill(s, 10)...)

0-dimensional Array{Int64,0}:
0

In [66]:
using Zygote
Zygote.gradient(x->ein"afl,bhn,cjf,dlh,enj,ago,big,cki,dmk,eom->"(x,s,s,s,s,s,s,s,s,s)[], s)

([0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]

[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]

[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0],)

In [67]:
ein"afl,bhn,cjf,dlh,enj,ago,big,cki,dmk,eom->"(ones(3,3,3),s,s,s,s,s,s,s,s,s)

0-dimensional Array{Float64,0}:
0.0

In [59]:
# naive looping
satisfied(a, b, c) = a!=b && b!=c && c!=a ? 1 : 0

function petersen_coloring()
    res = 0
    for a=1:3, b=1:3, c=1:3, d=1:3, e=1:3, f=1:3, g=1:3, h=1:3, i=1:3, j=1:3, k=1:3, l=1:3, m=1:3, n=1:3, o=1:3
        res += satisfied(a,f,l)*satisfied(b,h,n)*satisfied(c,j,f)*satisfied(d,l,h)*satisfied(e,n,j)*
        satisfied(a,g,o)*satisfied(b,i,g)*satisfied(c,k,i)*satisfied(d,m,k)*satisfied(e,o,m)
    end
    res
end

petersen_coloring()

0

# properties of s tensors

In [7]:
# define a delta tensor
function δ(n::Int)
    t = zeros(Int, fill(n, n) |> Tuple)
    for i=1:n
        t[fill(i, n)...] = 1
    end
    t
end

δ (generic function with 1 method)

In [8]:
using Test

In [14]:
@test ein"ijk->ikj"(s)≈ s

Test Passed

In [19]:
@test ein"ijk,ijk->"(s,s)[] ≈ 6

Test Passed

In [21]:
@test ein"ijk,klm->ijlm"(s,s) ≈
    ein"il,jm->ijlm"(I3,I3) +
    ein"im,jl->ijlm"(I3,I3) -
    2*ein"ijk,klm->ijlm"(δ(3),δ(3))

Test Passed

In [41]:
# idempotence
E = ein"ijk,lmn->ijklmn"(s, s)/factorial(3)
@test ein"ijklmn,lmnopq->ijkopq"(E, E) ≈ E

Test Passed

In [29]:
@test ein"ijk,jkl->il"(s, s) ≈ 2*I

Test Passed

In [39]:
@test ein"ijk,klm,mni->jln"(s, s, s) ≈ s

Test Passed

In [51]:
@test einsum(((1,2,3), (3,4,5), (5,6,7), (7,8,1)), (s, s, s, s), (2,4,6,8)) ≈
    einsum(((2,8),(4,6)), (I3,I3), (2,4,6,8)) +
    einsum(((2,4),(8,6)), (I3,I3), (2,4,6,8))

Test Passed

In [40]:
@test ein"ijk,klm,mno,opq,qsi->jlnps"(s, s, s, s, s) ≈
    ein"jln,ps->jlnps"(s,I3) +
    ein"jps,ln->jlnps"(s,I3) +
    ein"nps,jl->jlnps"(s,I3) -
    ein"sjt,tlu,unp->jlnps"(s,s,s)

Test Passed

# 3-SAT problem

Example: $(\lnot x_1 \lor x_2 \lor x_3)\land(\lnot x_2\lor\lnot x_3\lor x_4)\land(x_3\lor x_1\lor x_4)$

In [99]:
indices = CartesianIndices((2,2,2))
C1 = map(x-> Int(x[1]==1 || x[2]==2 || x[3]==2), indices)
C2 = map(x-> Int(x[1]==1 || x[2]==1 || x[3]==2), indices)
C3 = map(x-> Int(x[1]==2 || x[2]==2 || x[3]==2), indices);

In [100]:
ein"abc,bcd,cad->"(C1, C2, C3)

0-dimensional Array{Int64,0}:
10

In [103]:
sat(x1, x2, x3, x4) = any((x1,x2,x3) .== (0,1,1)) && any((x2,x3,x4) .== (0,0,1)) && any((x3,x1,x4) .== (1,1,1))
using BitBasis
for b in basis(4)
    x1 = readbit(b, 1)
    x2 = readbit(b, 2)
    x3 = readbit(b, 3)
    x4 = readbit(b, 4)
    if sat(x1,x2,x3,x4)
        println((b |> bitstring)[end-3:end])
    end
end

0011
0100
0101
1000
1010
1011
1100
1101
1110
1111
